# 🏇 NAR 全レース取得
以下の設定変数を変更して実行してください。NAR（地方競馬）のデータを日付順に取得します。

In [ ]:
# Google Driveをマウントする場合のみ実行してください
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
import re
import time
from datetime import datetime

class RaceScraper:
    def __init__(self):
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    def _get_soup(self, url):
        try:
            time.sleep(1) # Be polite
            response = requests.get(url, headers=self.headers, timeout=10)
            response.encoding = response.apparent_encoding
            if response.status_code == 200:
                return BeautifulSoup(response.text, 'html.parser')
        except Exception as e:
            print(f"Error fetching {url}: {e}")
        return None

    def get_past_races(self, horse_id, n_samples=5):
        """
        Fetches past n_samples race results for a given horse_id from netkeiba db.
        Returns a DataFrame of past races.
        """
        url = f"https://db.netkeiba.com/horse/result/{horse_id}/"
        soup = self._get_soup(url)
        if not soup:
            return pd.DataFrame()

        # The results are usually in a table with class "db_h_race_results"
        table = soup.select_one("table.db_h_race_results")
        if not table:
            # Try to find any table with "着順"
            tables = soup.find_all("table")
            for t in tables:
                if "着順" in t.text:
                    table = t
                    break
        
        if not table:
            return pd.DataFrame()

        # Parse table
        # We need to manually parse to get clean data and handle links if needed (though for past data, text is mostly fine)
        # pd.read_html is easier for the table
        try:
            df = pd.read_html(io.StringIO(str(table)))[0]
            
            # Basic cleaning
            df = df.dropna(how='all')
            
            # The columns in db.netkeiba are roughly:
            # 日付, 開催, 天気, R, レース名, 映像, 頭数, 枠番, ... 着順, ... 通過, ...
            
            # We want to keep: Date, Race Name, Course info, Rank, Time, Passing (Style)
            
            # Normalize column names (remove spaces/newlines)
            df.columns = df.columns.astype(str).str.replace(r'\s+', '', regex=True)

            # Filter rows that look like actual races (Date column exists)
            if '日付' in df.columns:
                df['date_obj'] = pd.to_datetime(df['日付'], format='%Y/%m/%d', errors='coerce')
                df = df.dropna(subset=['date_obj'])
                df = df.sort_values('date_obj', ascending=False)
                
            # Take top N
            if n_samples:
                df = df.head(n_samples)
            
            # Process Run Style (Leg Type)
            if '通過' in df.columns:
                df['run_style_val'] = df['通過'].apply(self.extract_run_style)
            else:
                df['run_style_val'] = 3 # Unknown

            # Extract/Rename Columns
            # We want: 日付, 開催, 天気, R, レース名, 映像, 頭数, 枠番, ... 着順, ... 通過, ...
            # Important: '上り' (3F), '馬体重', '騎手'
            
            # Map standard columns if they exist
            column_map = {
                '日付': 'date',
                '開催': 'venue',
                '天気': 'weather',
                'レース名': 'race_name',
                '着順': 'rank',
                '枠番': 'waku',
                '馬番': 'umaban',
                '騎手': 'jockey',
                '斤量': 'weight_carried',
                '馬場': 'condition', # 良/重/稍重 etc.
                'タイム': 'time',
                '着差': 'margin',
                '上り': 'last_3f',
                '通過': 'passing',
                '馬体重': 'horse_weight',
                'run_style_val': 'run_style',
                '単勝': 'odds',
                'オッズ': 'odds',
                '距離': 'raw_distance' # e.g. "芝1600"
            }
            
            # Rename available columns
            df.rename(columns=column_map, inplace=True)
            
            # Extract Surface and Distance from 'raw_distance'
            if 'raw_distance' in df.columns:
                def parse_dist(x):
                    if not isinstance(x, str): return None, None
                    # "芝1600", "ダ1200", "障3000"
                    # Sometimes "芝1600" or just "1600"
                    surf = None
                    dist = None
                    if '芝' in x: surf = '芝'
                    elif 'ダ' in x: surf = 'ダ'
                    elif '障' in x: surf = '障'
                    
                    # Extract number
                    match = re.search(r'(\d+)', x)
                    if match:
                        dist = int(match.group(1))
                    return surf, dist

                parsed = df['raw_distance'].apply(parse_dist)
                df['course_type'] = parsed.apply(lambda x: x[0])
                df['distance'] = parsed.apply(lambda x: x[1])
            else:
                df['course_type'] = None
                df['distance'] = None

            # Coerce numeric
            if 'rank' in df.columns:
                df['rank'] = pd.to_numeric(df['rank'], errors='coerce')
            
            if 'odds' in df.columns:
                 df['odds'] = pd.to_numeric(df['odds'], errors='coerce')
            
            # Fill missing
            for target_col in list(column_map.values()) + ['course_type', 'distance']:
                if target_col not in df.columns:
                    df[target_col] = None
                
            return df
            
        except Exception as e:
            print(f"Error parsing past races for {horse_id}: {e}")
            return pd.DataFrame()

    def extract_run_style(self, passing_str):
        """
        Converts passing order string (e.g., "1-1-1", "10-10-12") to run style (1,2,3,4).
        1: Nige (Escape) - Lead at 1st corner
        2: Senkou (Leader) - Within first ~4 or so
        3: Sashi (Mid) - Midpack
        4: Oikomi (Chaser) - Back
        Returns integer code.
        """
        if not isinstance(passing_str, str):
            return 3 # Default to Mid
            
        # Clean string "1-1-1" -> [1, 1, 1]
        try:
            cleaned = re.sub(r'[^0-9-]', '', passing_str)
            parts = [int(p) for p in cleaned.split('-') if p]
            
            if not parts:
                return 3
                
            first_corner = parts[0]
            
            # Heuristics
            if first_corner == 1:
                return 1 # Nige
            elif first_corner <= 4:
                return 2 # Senkou
            elif first_corner <= 9: # Assuming standard field size of 10-16, 9 is mid-ish limit? 
                # Actually "Sashi" is usually mid-rear. 
                # Let's say: 1=Lead, 2-4=Front, 5-10=Mid, >10=Back
                return 3 # Sashi
            else:
                return 4 # Oikomi
                
        except:
            return 3

    def scrape_race_with_history(self, race_id):
        """
        Detailed scraper that enters a race_result page, finding horse IDs, 
        then fetches history for each horse.
        Returns a dictionary or structured object with the race result + history.
        """
        url = f"https://race.netkeiba.com/race/result.html?race_id={race_id}"
        soup = self._get_soup(url)
        if not soup:
            return None
            
        # 1. Parse Main Result Table to get Horse IDs and basic result
        # Note: auto_scraper already does some of this, but we need Horse IDs specifically.
        # "All_Result_Table"
        
        result_data = []
        
        table = soup.find("table", id="All_Result_Table")
        if not table:
            return None
            
        rows = table.find_all("tr", class_="HorseList")
        
        print(f"Found {len(rows)} horses in race {race_id}. Fetching histories...")
        
        for row in rows:
            # Extract basic info
            rank_elem = row.select_one(".Rank")
            rank = rank_elem.text.strip() if rank_elem else ""
            
            horse_name_elem = row.select_one(".Horse_Name a")
            horse_name = horse_name_elem.text.strip() if horse_name_elem else ""
            horse_url = horse_name_elem.get("href") if horse_name_elem else ""
            
            # Extract ID from URL
            # https://db.netkeiba.com/horse/2018105027
            horse_id = None
            if horse_url:
                match = re.search(r'/horse/(\d+)', horse_url)
                if match:
                    horse_id = match.group(1)
            
            if not horse_id:
                print(f"  Skipping {horse_name} (No ID)")
                continue

            print(f"  Fetching history for {horse_name} ({horse_id})...")
            
            # 2. Get Past History
            df_past = self.get_past_races(horse_id, n_samples=5)
            
            # 3. Structure Data
            # converting df_past to a list of dicts or flattened fields
            history = []
            if not df_past.empty:
                for idx, r in df_past.iterrows():
                    # Extract relevant columns
                    # We need at least: Rank, RunStyle, Time(Seconds?), Pace?
                    # For now just dump raw-ish data
                    hist_item = {
                        "date": r.get('日付'),
                        "race_name": r.get('レース名'),
                        "rank": r.get('着順'),
                        "passing": r.get('通過'),
                        "run_style": r.get('run_style_val'),
                        "time": r.get('タイム'),
                        # Add more as needed for Feature Engineering
                    }
                    history.append(hist_item)
            
            entry = {
                "race_id": race_id,
                "horse_id": horse_id,
                "horse_name": horse_name,
                "rank": rank,
                "history": history
            }
            result_data.append(entry)
            
        return result_data

    def get_horse_profile(self, horse_id):
        """
        Fetches horse profile to get pedigree (Father, Mother, Grandfather(BMS)).
        Returns a dictionary or None.
        """
        # Use pedigree page for reliable bloodline data
        url = f"https://db.netkeiba.com/horse/ped/{horse_id}/"
        soup = self._get_soup(url)
        if not soup:
            return None
        
        # Parse Blood Table
        # table class="blood_table"
        
        data = {
            "father": "",
            "mother": "",
            "bms": ""
        }
        
        try:
            table = soup.select_one("table.blood_table")
            if table:
                rows = table.find_all("tr")
                # 5-generation table has 32 rows usually
                # Father at Row 0 (rowspan 16)
                # Mother at Row 16 (rowspan 16)
                
                if len(rows) >= 17:
                    # Father: Row 0, Col 0
                    r0 = rows[0].find_all("td")
                    if r0:
                        txt = r0[0].text.strip()
                        # Clean: "スクリーンヒーロー\n2004 栗毛..." -> "スクリーンヒーロー"
                        # Take first line
                        data["father"] = txt.split('\n')[0].strip()
                        
                    # Mother & BMS: Row 16
                    r16 = rows[16].find_all("td")
                    if len(r16) >= 2:
                        # Mother
                        m_txt = r16[0].text.strip()
                        data["mother"] = m_txt.split('\n')[0].strip()
                        
                        # BMS (Mother's Father)
                        bms_txt = r16[1].text.strip()
                        data["bms"] = bms_txt.split('\n')[0].strip()
                        
        except Exception as e:
            print(f"Error parsing profile for {horse_id}: {e}")
            
        return data

    def get_race_metadata(self, race_id):
        """
        Fetches metadata for a specific race ID from Netkeiba.
        Returns dict with: race_name, date, venue, course_type, distance, weather, condition, turn
        """
        url = f"https://race.netkeiba.com/race/result.html?race_id={race_id}"
        soup = self._get_soup(url)
        if not soup:
            return None
            
        data = {
            "race_name": "",
            "date": "",
            "venue": "",
            "course_type": "",
            "distance": "",
            "weather": "",
            "condition": "",
            "turn": "", # New: Dictionary key for turn direction
            "race_id": race_id
        }
        
        try:
            # Race Name
            title_elem = soup.select_one(".RaceName")
            if title_elem:
                data["race_name"] = title_elem.text.strip()
                
            # Date & Venue & Conditions
            # <div class="RaceData01">... 2023年1月5日 ... 1回中山1日 ...</div>
            # Content: "15:35発走 / 芝1600m (右 外) / 天候:晴 / 馬場:良"
            
            rd1 = soup.select_one(".RaceData01")
            
            if rd1:
                txt = rd1.text.strip()
                
                # Weather
                if "天候:晴" in txt: data["weather"] = "晴"
                elif "天候:曇" in txt: data["weather"] = "曇"
                elif "天候:小雨" in txt: data["weather"] = "小雨"
                elif "天候:雨" in txt: data["weather"] = "雨"
                elif "天候:雪" in txt: data["weather"] = "雪"
                
                # Condition
                if "馬場:良" in txt: data["condition"] = "良"
                elif "馬場:稍" in txt: data["condition"] = "稍重" # Covers 稍重
                elif "馬場:重" in txt: data["condition"] = "重"
                elif "馬場:不良" in txt: data["condition"] = "不良"
                
                # Course & Distance ("芝1600m")
                # Regex for "芝", "ダ", "障" followed by digits
                match = re.search(r'(芝|ダ|障)(\d+)m', txt)
                if match:
                    ctype_raw = match.group(1)
                    if ctype_raw == "芝": data["course_type"] = "芝"
                    elif ctype_raw == "ダ": data["course_type"] = "ダート"
                    elif ctype_raw == "障": data["course_type"] = "障害"
                    
                    data["distance"] = match.group(2)
                
                # Turn Direction ("右", "左", "直線")
                # Usually in parentheses like "(右)" or "(左)" or "(芝 左)"
                if "右" in txt: data["turn"] = "右"
                elif "左" in txt: data["turn"] = "左"
                elif "直線" in txt: data["turn"] = "直"

            # Date
            # Try finding date in Title or dedicated element
            date_elem = soup.select_one("dl#RaceList_DateList dd.Active") 
            if date_elem:
                 # Usually "1月5日(金)" - needs Year
                 # We can rely on the fact that race_id contains year (2025...)
                 # But let's look for YYYY年 in the whole text or title
                 pass
            
            # Fallback Date from Title Tag or Meta
            if not data["date"]:
                 meta_title = soup.title.text if soup.title else ""
                 match_date = re.search(r'(\d{4}年\d{1,2}月\d{1,2}日)', meta_title)
                 if match_date:
                     data["date"] = match_date.group(1)

        except Exception as e:
            print(f"Error parsing metadata for {race_id}: {e}")
            
        return data

if __name__ == "__main__":
    # Test
    scraper = RaceScraper()
    print("Running test...")
    # Example: Do Deuce (2019105219)
    # url = "https://db.netkeiba.com/horse/2019105219/"
    # print(f"Fetching {url}")
    df = scraper.get_past_races("2019105219")
    if df.empty:
        print("DF is empty. Checking raw soup for 'db_h_race_results'...")
        soup = scraper._get_soup(f"https://db.netkeiba.com/horse/result/2019105219/")
        if soup:
             t = soup.select_one("table.db_h_race_results")
             print(f"Selector 'table.db_h_race_results' found: {t is not None}")
             if not t:
                 print("Trying fallback 'table' with '着順'...")
                 tables = soup.find_all("table")
                 found = False
                 for i, tbl in enumerate(tables):
                     print(f"Table {i} classes: {tbl.get('class')}")
                     if "着順" in tbl.text or "着 順" in tbl.text or "日付" in tbl.text:
                         print("Found a table with '着順/日付'.")
                         # print(str(tbl)[:200])
                         t = tbl
                         found = True
                         break
                 if not found:
                     print("No table with '着順' found in soup.")
                     print("Soup snippet:", soup.text[:500])
                 else:
                    # Retry parsing with found table
                     try:
                        df = pd.read_html(str(t))[0]
                        print("Retry DF Head:")
                        print(df.head())
                     except Exception as e:
                        print(f"Retry parsing failed: {e}")
        else:
            print("Soup is None.")
    else:
        print(df.head())
        print("Columns:", df.columns)


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
import re
from datetime import datetime
import urllib.parse
import time

def scrape_jra_race(url, existing_race_ids=None):
    """
    Scrapes a single race page from JRA website.
    Returns a pandas DataFrame matching the schema of database.csv.
    If existing_race_ids is provided and the race ID is found, returns None (skip).
    """
    print(f"Accessing URL: {url}...")
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36" 
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        # Netkeiba usually uses EUC-JP, JRA uses Shift_JIS. 
        # Since this function is mostly used for Netkeiba (NAR/JRA-Backfill), default to EUC-JP.
        # response.encoding = 'cp932' 
        response.encoding = 'EUC-JP'
        
        if response.status_code != 200:
            print(f"Error: Status code {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # --- Metadata Extraction ---
        h1_elem = soup.select_one("div.header_line h1 .txt")
        full_text = h1_elem.text.strip() if h1_elem else ""
        if not full_text and soup.h1:
            full_text = soup.h1.text.strip()

        date_text = ""
        venue_text = ""
        race_num_text = ""
        kai = "01"
        day = "01"
        
        # Extract Date
        match_date = re.search(r'(\d{4}年\d{1,2}月\d{1,2}日)', full_text)
        if match_date:
            date_text = match_date.group(1)
            
        # Extract Venue, Kai, Day
        venues_str = "札幌|函館|福島|新潟|東京|中山|中京|京都|阪神|小倉"
        match_meta = re.search(rf'(\d+)回({venues_str})(\d+)日', full_text)
        if match_meta:
            kai = f"{int(match_meta.group(1)):02}"
            venue_text = match_meta.group(2)
            day = f"{int(match_meta.group(3)):02}"
            
        # Extract Race Num
        match_race = re.search(r'(\d+)レース', full_text)
        if match_race:
            r_val = int(match_race.group(1))
            race_num_text = f"{r_val}R"
            r_num = f"{r_val:02}"
        else:
            race_num_text = "10R" # Fallback
            r_num = "10"
            
        # Race Name
        race_name_text = ""
        name_elem = soup.select_one(".race_name")
        if name_elem:
            race_name_text = name_elem.text.strip()

        # Grade
        grade_text = ""
        if "G1" in str(soup) or "ＧⅠ" in str(soup): grade_text = "G1"
        elif "G2" in str(soup) or "ＧⅡ" in str(soup): grade_text = "G2"
        elif "G3" in str(soup) or "ＧⅢ" in str(soup): grade_text = "G3"

        # --- Added: Course, Distance, Weather, Condition ---
        # JRA HTML structure varies, but often contained in specific divs or text lines.
        # We will scan the entire header text or specific class for these patterns.
        
        # 1. Course & Distance (e.g., "芝2000メートル", "ダート1800メートル", "芝・1600m")
        # Usually in the same block as race name or just below.
        # We search the whole header area text.
        header_text = soup.select_one("div.header_line").text if soup.select_one("div.header_line") else soup.text
        
        # Regex: Allow spaces, dots, etc. between Type and Dist
        dist_type_match = re.search(r'(芝|ダ|ダート|障害)[^0-9]*(\d+)', header_text)
        course_type = ""
        distance = ""
        
        if dist_type_match:
            c_val = dist_type_match.group(1)
            d_val = dist_type_match.group(2)
            
            if "芝" in c_val: course_type = "芝"
            elif "ダ" in c_val: course_type = "ダート"
            elif "障" in c_val: course_type = "障害"
            
            distance = int(d_val)
        
        # 1.5 Rotation (Right/Left/Straight)
        rotation = ""
        # Often formatted as "(右)" or "（左）" 
        rot_match = re.search(r'[（\(](右|左|直線)[）\)]', header_text)
        if rot_match:
            rotation = rot_match.group(1)
        else:
             # Fallback: Inference based on Venue
             # Tokyo, Chukyo, Niigata -> Left (Default), others Right
             # Niigata 1000m -> Straight
             if "左" in header_text: rotation = "左"
             elif "右" in header_text: rotation = "右"
             elif "直線" in header_text: rotation = "直線"

        
        # 2. Weather (e.g., "天候：晴")
        weather = ""
        w_match = re.search(r'天候\s*[:：]\s*(\S+)', soup.text)
        if w_match:
            weather = w_match.group(1).strip()
            
        # 3. Condition (e.g., "芝：良", "ダート：稍重")
        # Note: A race can have both if it's mixed, but usually we care about the main one or the one matching course_type.
        condition = ""
        
        # Try specific pattern based on course type
        if course_type == "芝":
             c_match = re.search(r'芝\s*[:：]\s*(\S+)', soup.text)
             if c_match: condition = c_match.group(1).strip()
        elif course_type == "ダート":
             c_match = re.search(r'ダート\s*[:：]\s*(\S+)', soup.text)
             if c_match: condition = c_match.group(1).strip()
        
        # Fallback if generic or course type unknown, grab first one found
        if not condition:
             c_match_gen = re.search(r'(?:芝|ダート)\s*[:：]\s*(\S+)', soup.text)
             if c_match_gen: condition = c_match_gen.group(1).strip()

        # --- Table Extraction (Custom BS4 Parsing) ---
        # Find table with "着順"
        tables = soup.find_all('table')
        target_table = None
        for tbl in tables:
            if "着順" in tbl.text and "馬名" in tbl.text:
                target_table = tbl
                break
        
        if not target_table:
            print(f"Warning: Result table not found in {url} (Encoding: {response.encoding})")
            return None

        # Parse Rows
        rows = target_table.find_all('tr')
        data = []
        
        for row in rows:
            # Skip header (usually th) or invalid rows
            if row.find('th'):
                continue
                
            cells = row.find_all('td')
            if not cells:
                continue

            # Need to robustly map cells. 
            # We can use class names if available, or index.
            # Based on debug:
            # 0: place (着順)
            # 1: waku (枠) -> img alt
            # 2: num (馬番)
            # 3: horse (馬名)
            # 4: age (性齢)
            # 5: weight (斤量)
            # 6: jockey (騎手)
            # 7: time (タイム)
            # 8: margin (着差)
            # 9: corner (通過)
            # 10: f_time (上り)
            # 11: h_weight (馬体重)
            # 12: trainer (調教師)
            # 13: pop (人気)
            # * Odds is missing *

            def get_text(idx):
                if idx < len(cells):
                    return cells[idx].get_text(strip=True)
                return ""

            # Extract Frame (Waku) from Image
            waku_text = ""
            if len(cells) > 1:
                img = cells[1].find('img')
                if img and 'alt' in img.attrs:
                    # Example: "枠6緑" -> Extract number
                    alt = img['alt']
                    m = re.search(r'枠(\d+)', alt)
                    if m:
                        waku_text = m.group(1)
                    else:
                        waku_text = alt # Fallback
            
            # Extract Horse ID
            horse_id = ""
            if len(cells) > 3:
                a_tag = cells[3].find('a')
                if a_tag and 'href' in a_tag.attrs:
                    href = a_tag['href']
                    # /horse/2018105247/
                    m = re.search(r'/horse/(\d+)', href)
                    if m:
                        horse_id = m.group(1)

            row_data = {
                '着 順': get_text(0),
                '枠': waku_text,
                '馬 番': get_text(2),
                '馬名': get_text(3),
                'horse_id': horse_id, # Added
                '性齢': get_text(4),
                '斤量': get_text(5),
                '騎手': get_text(6),
                'タイム': get_text(7),
                '着差': get_text(8),
                'コーナー 通過順': get_text(9),
                '後3F': get_text(10),
                '馬体重 (増減)': get_text(11),
                '厩舎': get_text(12),
                '人 気': get_text(13),
                '単勝 オッズ': "0.0" # Missing in source
            }
            data.append(row_data)

        df = pd.DataFrame(data)
        
        # Add Metadata Columns
        df['日付'] = date_text
        df['会場'] = venue_text
        df['レース番号'] = race_num_text
        df['レース名'] = race_name_text
        df['重賞'] = grade_text
        df['距離'] = distance
        df['コースタイプ'] = course_type
        df['天候'] = weather
        df['天候'] = weather
        df['馬場状態'] = condition
        df['回り'] = rotation
        
        # ID Generation
        place_map = {
            "札幌": "01", "函館": "02", "福島": "03", "新潟": "04", "東京": "05",
            "中山": "06", "中京": "07", "京都": "08", "阪神": "09", "小倉": "10"
        }
        p_code = place_map.get(venue_text, "00")
        
        year = "2025"
        if date_text:
            year = date_text[:4]
            
        generated_id = f"{year}{p_code}{kai}{day}{r_num}"
        
        # SKIP CHECK
        if existing_race_ids and generated_id in existing_race_ids:
            print(f"Skipping {generated_id} (Already exists)")
            return None

        df['race_id'] = generated_id

        # Cleanups
        if '単勝 オッズ' in df.columns:
            df['単勝 オッズ'] = pd.to_numeric(df['単勝 オッズ'], errors='coerce').fillna(0.0)

        standard_columns = [
            "日付","会場","レース番号","レース名","重賞","着 順","枠","馬 番","馬名","性齢","斤量","騎手",
            "タイム","着差","人 気","単勝 オッズ","後3F","コーナー 通過順","厩舎","馬体重 (増減)","race_id",
            "距離","コースタイプ","天候","馬場状態","回り"
        ]
        
        for col in standard_columns:
            if col not in df.columns:
                df[col] = ""
                
        df = df[standard_columns]
        
        print(f"Scraped {len(df)} rows.")
        return df

    except Exception as e:
        print(f"Error scraping JRA URL: {e}")
        return None


# Parameter Map for Monthly Results (Reverse Engineered)
JRA_MONTH_PARAMS = {
    "2026": { "01": "E4", "02": "B2", "03": "80", "04": "4E", "05": "1C", "06": "EA", "07": "B8", "08": "86", "09": "54", "10": "22", "11": "F0", "12": "BE" },
    "2025": { "01": "3F", "02": "0D", "03": "DB", "04": "A9", "05": "77", "06": "45", "07": "13", "08": "E1", "09": "AF", "10": "1E", "11": "EC", "12": "D3" },
    "2024": { "01": "B3", "02": "81", "03": "4F", "04": "1D", "05": "EB", "06": "B9", "07": "87", "08": "55", "09": "23", "10": "92", "11": "60", "12": "2E" },
    "2023": { "01": "27", "02": "F5", "03": "C3", "04": "91", "05": "5F", "06": "2D", "07": "FB", "08": "C9", "09": "97", "10": "06", "11": "D4", "12": "A2" },
    "2022": { "01": "9B", "02": "69", "03": "37", "04": "05", "05": "D3", "06": "A1", "07": "6F", "08": "3D", "09": "0B", "10": "7A", "11": "48", "12": "16" },
    "2021": { "01": "0F", "02": "DD", "03": "AB", "04": "79", "05": "47", "06": "15", "07": "E3", "08": "B1", "09": "7F", "10": "EE", "11": "BC", "12": "8A" },
    "2020": { "01": "83", "02": "51", "03": "1F", "04": "ED", "05": "BB", "06": "89", "07": "57", "08": "25", "09": "F3", "10": "62", "11": "30", "12": "FE" }
}

def scrape_jra_year(year_str, start_date=None, end_date=None, save_callback=None, existing_race_ids=None):
    """
    Scrapes races for a given year and date range.
    year_str: "2024" or "2025"
    start_date: datetime.date (optional)
    end_date: datetime.date (optional)
    save_callback: function(df) to save progress
    """
    
    if year_str not in JRA_MONTH_PARAMS:
        print(f"Year {year_str} not supported in parameter map.")
        return

    params = JRA_MONTH_PARAMS[year_str]
    base_url = "https://www.jra.go.jp/JRADB/accessS.html"
    
    # Determine months to iterate
    start_m = 1
    end_m = 12
    
    if start_date:
        start_m = start_date.month
    if end_date:
        end_m = end_date.month
    
    # Cap at Today to prevent future scraping
    from datetime import date
    today = date.today()
    
    # If explicit end_date is used, respect it, but also respect today if it is earlier?
    # Usually for results, we never want future.
    if end_date:
        actual_end_date = min(end_date, today)
    else:
        actual_end_date = today
        
    print(f"=== Starting JRA Bulk Scraping for {year_str} (Period: {start_date or 'Start'} - {actual_end_date}) ===")

    # Adjust end_m based on today if we are in target year
    if int(year_str) == today.year:
        end_m = min(end_m, today.month)
    elif int(year_str) > today.year:
        print(f"Year {year_str} is in the future. Stopping.")
        return
    
    for m in range(start_m, end_m + 1):
        month = f"{m:02}"
        if month not in params:
            continue
            
        suffix = params[month]
        # Logic for skl00 vs skl10
        try:
            ym = int(year_str + month)
            prefix = "pw01skl00" if ym >= 202512 else "pw01skl10"
        except:
            prefix = "pw01skl10"

        cname = f"{prefix}{year_str}{month}/{suffix}"
        
        print(f"Fetching list for {year_str}/{month} (CNAME={cname})...")
        
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36" 
            }
            response = requests.post(base_url, data={"cname": cname}, headers=headers, timeout=10)
            response.encoding = 'cp932'
            
            if response.status_code != 200:
                print(f"Failed to fetch {cname} (Status {response.status_code})")
                continue
                
            soup = BeautifulSoup(response.text, 'html.parser')
            
            race_cnames = []
            links = soup.find_all('a')
            for link in links:
                onclick = link.get('onclick', '')
                match = re.search(r"doAction\('[^']+',\s*'([^']+)'\)", onclick)
                if match:
                    c = match.group(1)
                    if c.startswith('pw01srl'):
                        race_cnames.append(c)
            
            race_cnames = sorted(list(set(race_cnames)))
            print(f"  Found {len(race_cnames)} race days in month.")
            
            for day_cname in race_cnames:
                resp_day = requests.post(base_url, data={"cname": day_cname}, headers=headers, timeout=10)
                resp_day.encoding = 'cp932'
                soup_day = BeautifulSoup(resp_day.text, 'html.parser')
                
                # Check date of this day page
                day_date_text = ""
                d_h1 = soup_day.select_one("div.header_line h1 .txt")
                full_d_text = d_h1.text.strip() if d_h1 else (soup_day.h1.text.strip() if soup_day.h1 else "")
                
                # Parse date from "2025年1月5日（日曜）1回中山1日"
                # Need to match Date AND Venue/Kai/Day info for ID generation
                # Pattern: YYYY年M月D日 ... K回VenueD日
                
                current_day_date = None
                kai_str = "01"
                day_str = "01"
                venue_str = ""
                p_code = "00"
                
                match_day_date = re.search(r'(\d{4})年(\d{1,2})月(\d{1,2})日', full_d_text)
                if match_day_date:
                    y, mo, d_day = map(int, match_day_date.groups())
                    current_day_date = datetime(y, mo, d_day).date()
                    
                    # Filtering
                    if start_date and current_day_date < start_date:
                        print(f"    Skipping day {current_day_date} (Before start date)")
                        continue
                    if end_date and current_day_date > end_date:
                        print(f"    Skipping day {current_day_date} (After end date)")
                        continue
                    print(f"    Processing Day: {current_day_date} ({full_d_text})")
                else:
                    print(f"    Processing Day (Date unknown): {full_d_text[:20]}...")

                # Parse Venue Info for ID Generation
                venues_ptn = "札幌|函館|福島|新潟|東京|中山|中京|京都|阪神|小倉"
                match_meta = re.search(rf'(\d+)回({venues_ptn})(\d+)日', full_d_text)
                if match_meta:
                    kai_str = f"{int(match_meta.group(1)):02}"
                    venue_str = match_meta.group(2)
                    day_str = f"{int(match_meta.group(3)):02}"
                    
                    place_map = {
                        "札幌": "01", "函館": "02", "福島": "03", "新潟": "04", "東京": "05",
                        "中山": "06", "中京": "07", "京都": "08", "阪神": "09", "小倉": "10"
                    }
                    p_code = place_map.get(venue_str, "00")
                
                # Collect Race Links AND Race Numbers
                # Need to pair Link with Race Number
                race_list_items = []

                all_anchors = soup_day.find_all('a')
                for a in all_anchors:
                    onclick = a.get('onclick', '')
                    # Check for doAction with robust regex (handles single/double quotes, whitespace)
                    # Pattern: doAction('FormName', 'CNAME')
                    match_sde = re.search(r"doAction\s*\(\s*['\"][^'\"]+['\"]\s*,\s*['\"](pw01sde[^'\"]+)['\"]\s*\)", onclick)
                    href = a.get('href', '')
                    
                    final_url = ""
                    if match_sde:
                        final_url = f"{base_url}?CNAME={match_sde.group(1)}"
                    elif 'pw01sde' in href:
                        # Fallback for simple hrefs
                        if 'CNAME=' in href:
                             final_url = urllib.parse.urljoin(base_url, href)
                        else:
                             # If href="accessS.html?CNAME=..."
                             # or just "?CNAME=..."
                             final_url = urllib.parse.urljoin(base_url, href)
                    
                    if final_url:
                        # Extract Race Number from anchor text (e.g. "1R", "11R")
                        # Or generic image alt?
                        # Usually text is "1R" or img alt="1R"
                        txt = a.text.strip()
                        img = a.find('img')
                        if not txt and img and 'alt' in img.attrs:
                            txt = img['alt']
                        
                        r_num = -1
                        r_num_match = re.search(r'(\d+)R', txt)
                        if r_num_match:
                             r_num = int(r_num_match.group(1))
                             
                        # Append even if Race Num is not found (fix for missing races)
                        race_list_items.append((final_url, r_num))
                
                # Deduplicate by URL (keep first found usually fine)
                # Sort by Race Number (unknowns (-1) first or last?)
                seen_urls = set()
                unique_races = []
                for url, r_num in race_list_items:
                    if url not in seen_urls:
                        unique_races.append((url, r_num))
                        seen_urls.add(url)
                
                unique_races.sort(key=lambda x: x[1]) # Sort by race num (-1 will be first)
                
                print(f"      -> {len(unique_races)} races found.")

                for r_link, r_num in unique_races:
                    # PRE-FETCH OPTIMIZATION
                    # Construct ID
                    # Only if we successfully extracted Race Num and Venue info
                    if r_num != -1 and p_code != "00" and current_day_date:
                         # ID: YYYY PP KK DD RR
                         # y is from match_day_date loop var (int)
                         # p_code, kai_str, day_str strings
                         
                         # Ensure year is from the day page date
                         y_str = str(y)
                         r_num_str = f"{r_num:02}"
                         
                         generated_id = f"{y_str}{p_code}{kai_str}{day_str}{r_num_str}"
                         
                         if existing_race_ids and generated_id in existing_race_ids:
                             # print(f"        [Skip] {generated_id} (Pre-check)")
                             continue
                    
                    # If not skipped, fetch
                    df = scrape_jra_race(r_link, existing_race_ids=existing_race_ids)
                    
                    if df is not None and not df.empty:
                        if save_callback:
                            save_callback(df)
                        time.sleep(1)
        
        except Exception as e:
            print(f"Error processing month {month}: {e}")



In [ ]:
# NAR スクレイピングロジック
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import date, timedelta
import calendar
import time
import os

def run_nar_scraping(year, start_month=1, end_month=12, save_dir='data/raw'):
    start_date = date(int(year), int(start_month), 1)
    last_day = calendar.monthrange(int(year), int(end_month))[1]
    end_date = date(int(year), int(end_month), last_day)
    
    today = date.today()
    if end_date > today: end_date = today
    
    print(f'NARデータを {start_date} から {end_date} まで取得します...')
    print(f'保存先: {os.path.join(save_dir, "database_nar.csv")}')
    
    curr = start_date
    # scraper = RaceScraper() # Not used directly if we use scrape_jra_race
    
    while curr <= end_date:
        d_str = curr.strftime('%Y%m%d')
        url = f'https://nar.netkeiba.com/top/race_list_sub.html?kaisai_date={d_str}'
        try:
             time.sleep(0.5)
             headers = {'User-Agent': 'Mozilla/5.0'}
             resp = requests.get(url, headers=headers)
             resp.encoding = 'EUC-JP'
             soup = BeautifulSoup(resp.text, 'html.parser')
             links = soup.select('a[href*="race/result.html"]')
             
             if links:
                 print(f'{curr}: {len(links)} 件のレースが見つかりました。')
                 for link in links:
                     href = link.get('href')
                     if href.startswith('../'):
                         full_url = f'https://nar.netkeiba.com/{href.replace("../", "")}'
                     elif href.startswith('http'):
                         full_url = href
                     else:
                         full_url = f'https://nar.netkeiba.com{href}'
                     
                     # scrape_jra_race is compatible with netkeiba structure
                     try:
                         df = scrape_jra_race(full_url, existing_race_ids=None)
                         if df is not None and not df.empty:
                             # Save immediately
                             os.makedirs(save_dir, exist_ok=True)
                             mode = 'a'
                             csv_file = os.path.join(save_dir, 'database_nar.csv')
                             header = not os.path.exists(csv_file)
                             df.to_csv(csv_file, mode=mode, header=header, index=False)
                         time.sleep(1)
                     except Exception as e_race:
                         print(f'  Error scraping race {full_url}: {e_race}')
             else:
                 # print(f'{curr}: レースなし')
                 pass
        except Exception as e: print(e)
        curr += timedelta(days=1)


In [ ]:
# 設定 (ここを変更してください)
YEAR = 2024          # 対象年度
START_MONTH = 1      # 開始月
END_MONTH = 12       # 終了月
SAVE_DIR = '/content/drive/MyDrive/dai-keiba/data/raw' # 保存先フォルダ

# 実行ブロック
if YEAR:
    # ディレクトリ作成
    os.makedirs(SAVE_DIR, exist_ok=True)
    run_nar_scraping(YEAR, START_MONTH, END_MONTH, save_dir=SAVE_DIR)
